In [1]:
import os
import warnings

warnings.simplefilter(action="ignore")
os.environ["GRPC_VERBOSITY"] = "NONE"

# Prerequisites

Please make sure your environmental variables and dependencies are ready to use LLM services. Name of the environmental variables is arbitraray because langrila modules accepts that name as an argument.

In [2]:
from dotenv import load_dotenv

load_dotenv("../../.env_api")

True

# Import modules

In [3]:
from langrila import Agent, Tool
from langrila.anthropic import AnthropicClient
from langrila.google import GoogleClient
from langrila.openai import OpenAIClient

# Define tools

In this example, we use dummy tools as shown below. In langrila, tool's schema is generated from the tool's type hint and docstring in default, so it's important to write in details. Here is an example using dummy tools.

In [4]:
import random
from typing import Literal


def start_music() -> str:
    """
    Turn on the music. The genre, BPM, and volume are randomly selected.

    Returns
    ----------
    str
        A message indicating that the music is starting.
    """
    music_genre = random.choice(["rock", "pop", "jazz", "classical", "hip-hop"])
    volume = random.uniform(0.2, 1.0)
    bpm = 120
    return f"Starting music! Genre: {music_genre}, BPM: {bpm}, Volume: {volume}"


def change_bpm(new_bpm: int) -> int:
    """
    Change the BPM of the music.

    Parameters
    ----------
    new_bpm : int
        The new BPM to set the music.

    Returns
    ----------
    int
        The new BPM of the music.
    """
    return f"Changing BPM to {new_bpm}"


def change_music(
    genre: Literal["rock", "pop", "jazz", "classical", "hip-hop"],
    bpm: int,
) -> str:
    """
    Change the music genre and BPM.

    Parameters
    ----------
    genre : str
        The genre of music to play. Should be one of "rock", "pop", "jazz", "classical", or "hip-hop".
    bpm : int
        The BPM of the music.

    Returns
    ----------
    str
        A message indicating that the music has been changed.
    """
    return f"Changing music to {genre} with BPM {bpm}"

# Instantiating Agent

Client modules

In [5]:
# For OpenAI
openai_client = OpenAIClient(api_key_env_name="OPENAI_API_KEY")

# For Azure OpenAI
azure_openai_client = OpenAIClient(
    api_key_env_name="AZURE_API_KEY",
    api_type="azure",
    azure_api_version="2024-11-01-preview",
    azure_endpoint_env_name="AZURE_ENDPOINT",
    azure_deployment_id_env_name="AZURE_DEPLOYMENT_ID",
)

# For Gemini on Google AI Studio
google_dev_client = GoogleClient(
    api_key_env_name="GEMINI_API_KEY",
)

# For Gemini on Google Cloud VertexAI
vertexai_client = GoogleClient(
    api_type="vertexai",
    project_id_env_name="GOOGLE_CLOUD_PROJECT",
    location="us-central1",
)

# For Claude of Anthropic
anthropic_client = AnthropicClient(
    api_key_env_name="ANTHROPIC_API_KEY",
)

# For Claude of Amazon Bedrock
claude_bedrock_client = AnthropicClient(
    api_type="bedrock",
    aws_access_key_env_name="AWS_ACCESS_KEY",
    aws_secret_key_env_name="AWS_SECRET_KEY",
    aws_region_env_name="AWS_REGION",
)

Tools must be specified when the agent is initialized.

In [6]:
tools = [start_music, change_bpm, change_music]

openai_agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
    tools=tools,
)

# Generating text with tool calling

## Synchronous, non-streaming text generation

In [7]:
prompt = "Play some music, well, prefer to jazz music with 80 BPM."

response = openai_agent.generate_text(prompt)

print(response.contents[0].text)

[2025-01-02 19:21:15] DEBUG | Prompt: [TextPrompt(text='Play some music, well, prefer to jazz music with 80 BPM.')]
[2025-01-02 19:21:15] INFO | root: Generating text
[2025-01-02 19:21:16] DEBUG | Response: [ToolCallResponse(name='change_music', args='{"genre":"jazz","bpm":80}', call_id='call_dnFYQdXHcVMcP5VgvvJf00sE')]
[2025-01-02 19:21:16] INFO | Running tool: change_music
[2025-01-02 19:21:16] INFO | Tool: change_music successfully ran.
[2025-01-02 19:21:16] DEBUG | Prompt: [ToolUsePrompt(output='Changing music to jazz with BPM 80', error=None, call_id='call_dnFYQdXHcVMcP5VgvvJf00sE', args='{"genre":"jazz","bpm":80}', name='change_music')]
[2025-01-02 19:21:16] INFO | root: Generating text
[2025-01-02 19:21:16] DEBUG | Response: [TextResponse(text="I've changed the music to jazz with a BPM of 80. Enjoy the tunes!")]


I've changed the music to jazz with a BPM of 80. Enjoy the tunes!


Agent finally generates text response. Steps the agent generates final answer is like: 

1. Feed user's input
2. Call tools
3. Validate args with the pydantic schema validator
4. Actually run called tools
5. Generate text response based on the tool calling results

BTW, In this case, the `change_music` tool was called before the `start_music` tool. Those are all dummy tools so it can be run in any order, but this is unexpected behaviour. To resolve such invalid tool calling, you can specify `planning` argument when the agent is instantiated. If `planning` is True, the agent makes a plan to answer the user's input based on the given tools, and then generates answer.

In [ ]:
openai_agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
    tools=tools,
    planning=True,  # <- Enable planning
)

In [10]:
response = openai_agent.generate_text(prompt)

print(response.contents[0].text)

[2025-01-02 19:22:25] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, prefer to jazz music with 80 BPM.\n\nCapabilities:\n- start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n- change_bpm: Change the BPM of the music.\n- change_music: Change the music genre and BPM.\n')]
[2025-01-02 19:22:25] INFO | root: Generating text
[2025-01-02 19:22:28] DEBUG | Response: [TextResponse(text='### Plan to Fulfill the Requirement\n\n1. **Start Music**: Invoke the `start_music` tool to begin playing music. This will randomly select a genre and BPM.\n2. **Check BPM**: If the BPM is not 80, use the `change_bpm` tool to adjust the BPM to 80.\n3. **Check Genre

The music has been started with the genre set to rock and BPM at 120. I have successfully changed the music to jazz and adjusted the BPM to 80. Enjoy the jazz music!


By planning, `start_music` tool was called at the first step, then `change_bpm ` tool and `change_music` tool was.

Here is the default planning prompt.

In [11]:
from langrila import AgentConfig

print(AgentConfig().internal_prompt.planning)

Please make a concise plan to answer the following question/requirement, considering the conversation history.
You can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.
Agent has no description while the tools have a description.

Question/Requirement:
{user_input}

Capabilities:
{capabilities}


This AgentConfig is configurable and customized config can be passes to the agent when instantiating as mentioned in [01.introduction.ipynb](./01.introduction.ipynb).

## Asynchronous, non-streaming text generation

In [12]:
response = await openai_agent.generate_text_async(prompt)

print(response.contents[0].text)

[2025-01-02 19:29:58] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, prefer to jazz music with 80 BPM.\n\nCapabilities:\n- start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n- change_bpm: Change the BPM of the music.\n- change_music: Change the music genre and BPM.\n')]
[2025-01-02 19:29:58] INFO | root: Generating text
[2025-01-02 19:30:01] DEBUG | Response: [TextResponse(text='### Plan to Answer the Requirement\n\n1. **Start Music**: Invoke the `start_music` tool to begin playing music. This will randomly select a genre and BPM.\n2. **Check BPM**: If the BPM is not 80, use the `change_bpm` tool to adjust the BPM to 80.\n3. **Check Genre*

The music has been started with a classical genre at 120 BPM. I have successfully changed the music to jazz with a BPM of 80. Enjoy the music!


## Synchronous, streaming text generation

In [13]:
from langrila import TextResponse, ToolCallResponse
# from langrila.core.logger import DEFAULT_LOGGER as default_logger

In [15]:
streamed_response = openai_agent.stream_text(prompt)

for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-02 19:30:33] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, prefer to jazz music with 80 BPM.\n\nCapabilities:\n- start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n- change_bpm: Change the BPM of the music.\n- change_music: Change the music genre and BPM.\n')]
[2025-01-02 19:30:33] INFO | root: Generating text
[2025-01-02 19:30:36] DEBUG | Response: [TextResponse(text='### Plan to Fulfill the Requirement\n\n1. **Start Music**: Invoke the `start_music` tool to begin playing music. This will randomly select a genre and BPM.\n2. **Check BPM**: If the BPM is not 80, use the `change_bpm` tool to adjust the BPM to 80.\n3. **Check Genre

{}


[2025-01-02 19:30:37] DEBUG | Response: [ToolCallResponse(name='start_music', args='{}', call_id='call_YC36KgNcWqVG1KZGITTjlKYV')]
[2025-01-02 19:30:37] INFO | Running tool: start_music
[2025-01-02 19:30:37] INFO | Tool: start_music successfully ran.
[2025-01-02 19:30:37] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: rock, BPM: 120, Volume: 0.5247184194788775', error=None, call_id='call_YC36KgNcWqVG1KZGITTjlKYV', args='{}', name='start_music')]
[2025-01-02 19:30:37] INFO | root: Generating text


{"ne
{"new_bpm
{"new_bpm": 80}
{"ge
{"genre":
{"genre": "jazz
{"genre": "jazz", "
{"genre": "jazz", "bpm":
{"genre": "jazz", "bpm": 80}


[2025-01-02 19:30:38] DEBUG | Response: [ToolCallResponse(name='change_bpm', args='{"new_bpm": 80}', call_id='call_8ZeMt8HKJpY7NxtTLL9JXRV1'), ToolCallResponse(name='change_music', args='{"genre": "jazz", "bpm": 80}', call_id='call_WyIMjtuC4dkuEXahI14WYUdr')]
[2025-01-02 19:30:38] INFO | Running tool: change_bpm
[2025-01-02 19:30:38] INFO | Tool: change_bpm successfully ran.
[2025-01-02 19:30:38] INFO | Running tool: change_music
[2025-01-02 19:30:38] INFO | Tool: change_music successfully ran.
[2025-01-02 19:30:38] DEBUG | Prompt: [ToolUsePrompt(output='Changing BPM to 80', error=None, call_id='call_8ZeMt8HKJpY7NxtTLL9JXRV1', args='{"new_bpm": 80}', name='change_bpm'), ToolUsePrompt(output='Changing music to jazz with BPM 80', error=None, call_id='call_WyIMjtuC4dkuEXahI14WYUdr', args='{"genre": "jazz", "bpm": 80}', name='change_music')]
[2025-01-02 19:30:38] INFO | root: Generating text


The
The music
The music has
The music has been
The music has been started
The music has been started with
The music has been started with the
The music has been started with the genre
The music has been started with the genre rock
The music has been started with the genre rock and
The music has been started with the genre rock and BPM
The music has been started with the genre rock and BPM 
The music has been started with the genre rock and BPM 120
The music has been started with the genre rock and BPM 120.
The music has been started with the genre rock and BPM 120. I
The music has been started with the genre rock and BPM 120. I have
The music has been started with the genre rock and BPM 120. I have successfully
The music has been started with the genre rock and BPM 120. I have successfully changed
The music has been started with the genre rock and BPM 120. I have successfully changed the
The music has been started with the genre rock and BPM 120. I have successfully changed the music
T

[2025-01-02 19:30:40] DEBUG | Response: [TextResponse(text='The music has been started with the genre rock and BPM 120. I have successfully changed the music to jazz with a BPM of 80. Enjoy the music!')]


The music has been started with the genre rock and BPM 120. I have successfully changed the music to jazz with a BPM of 80. Enjoy the music!


In [16]:
streamed_response = openai_agent.stream_text_async(prompt)

async for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-02 19:30:49] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, prefer to jazz music with 80 BPM.\n\nCapabilities:\n- start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n- change_bpm: Change the BPM of the music.\n- change_music: Change the music genre and BPM.\n')]
[2025-01-02 19:30:49] INFO | root: Generating text
[2025-01-02 19:30:52] DEBUG | Response: [TextResponse(text='### Plan to Fulfill the Requirement\n\n1. **Start Music**: Invoke the `start_music` tool to begin playing music. This will randomly select a genre and BPM.\n2. **Check BPM**: If the BPM is not 80, use the `change_bpm` tool to adjust the BPM to 80.\n3. **Check Genre

{}


[2025-01-02 19:30:52] DEBUG | Response: [ToolCallResponse(name='start_music', args='{}', call_id='call_aC1ym2HICOzQfqiiv0gN4UhW')]
[2025-01-02 19:30:52] INFO | Running tool: start_music
[2025-01-02 19:30:52] INFO | Tool: start_music successfully ran.
[2025-01-02 19:30:52] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: pop, BPM: 120, Volume: 0.55989089434875', error=None, call_id='call_aC1ym2HICOzQfqiiv0gN4UhW', args='{}', name='start_music')]
[2025-01-02 19:30:52] INFO | root: Generating text


{"ne
{"new_bpm
{"new_bpm": 80}
{"ge
{"genre":
{"genre": "jazz
{"genre": "jazz", "
{"genre": "jazz", "bpm":
{"genre": "jazz", "bpm": 80}


[2025-01-02 19:30:54] DEBUG | Response: [ToolCallResponse(name='change_bpm', args='{"new_bpm": 80}', call_id='call_rKjK2Oi0qGj7IEuQhJq2s8xh'), ToolCallResponse(name='change_music', args='{"genre": "jazz", "bpm": 80}', call_id='call_97YpFvdxmp5ul9PpqDC1ZZCg')]
[2025-01-02 19:30:54] INFO | Running tool: change_bpm
[2025-01-02 19:30:54] INFO | Tool: change_bpm successfully ran.
[2025-01-02 19:30:54] INFO | Running tool: change_music
[2025-01-02 19:30:54] INFO | Tool: change_music successfully ran.
[2025-01-02 19:30:54] DEBUG | Prompt: [ToolUsePrompt(output='Changing BPM to 80', error=None, call_id='call_rKjK2Oi0qGj7IEuQhJq2s8xh', args='{"new_bpm": 80}', name='change_bpm'), ToolUsePrompt(output='Changing music to jazz with BPM 80', error=None, call_id='call_97YpFvdxmp5ul9PpqDC1ZZCg', args='{"genre": "jazz", "bpm": 80}', name='change_music')]
[2025-01-02 19:30:54] INFO | root: Generating text


The
The music
The music has
The music has been
The music has been started
The music has been started with
The music has been started with a
The music has been started with a genre
The music has been started with a genre of
The music has been started with a genre of pop
The music has been started with a genre of pop and
The music has been started with a genre of pop and a
The music has been started with a genre of pop and a BPM
The music has been started with a genre of pop and a BPM of
The music has been started with a genre of pop and a BPM of 
The music has been started with a genre of pop and a BPM of 120
The music has been started with a genre of pop and a BPM of 120.
The music has been started with a genre of pop and a BPM of 120. I
The music has been started with a genre of pop and a BPM of 120. I have
The music has been started with a genre of pop and a BPM of 120. I have successfully
The music has been started with a genre of pop and a BPM of 120. I have successfully changed
Th

[2025-01-02 19:30:55] DEBUG | Response: [TextResponse(text='The music has been started with a genre of pop and a BPM of 120. I have successfully changed the BPM to 80 and the genre to jazz. Enjoy the music!')]


The music has been started with a genre of pop and a BPM of 120. I have successfully changed the BPM to 80 and the genre to jazz. Enjoy the music!


## Other client

Other clients has the same interface.

In [17]:
# Azure OpenAI
azure_openai_agent = Agent(
    client=azure_openai_client,
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
    tools=tools,
    planning=True,
)

# Google AI Studio
google_agent = Agent(
    client=google_dev_client,
    model="gemini-2.0-flash-exp",
    temperature=0.0,
    tools=tools,
    planning=True,
)

# Google Cloud VertexAI
vertexai_agent = Agent(
    client=vertexai_client,
    model="gemini-2.0-flash-exp",
    temperature=0.0,
    tools=tools,
    planning=True,
)

# Anthropic Claude
claude_agent = Agent(
    client=anthropic_client,
    model="claude-3-5-sonnet-20240620",
    temperature=0.0,
    max_tokens=500,
    tools=tools,
    planning=True,
)

# Claude on Amazon Bedrock
claude_bedrock_agent = Agent(
    client=claude_bedrock_client,
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    temperature=0.0,
    max_tokens=500,
    tools=tools,
    planning=True,
)

### Azure OpenAI

In [18]:
response = azure_openai_agent.generate_text(prompt)

print(response.contents[0].text)

[2025-01-02 19:31:07] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, prefer to jazz music with 80 BPM.\n\nCapabilities:\n- start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n- change_bpm: Change the BPM of the music.\n- change_music: Change the music genre and BPM.\n')]
[2025-01-02 19:31:07] INFO | root: Generating text
[2025-01-02 19:31:10] DEBUG | Response: [TextResponse(text='### Plan to Fulfill the Requirement\n\n1. **Start Music**: Invoke the `start_music` tool to begin playing music. This will randomly select a genre and BPM.\n2. **Check BPM**: If the BPM is not 80, use the `change_bpm` tool to adjust the BPM to 80.\n3. **Check Genre

The music has been started with a genre of pop and a BPM of 120. I have successfully changed the BPM to 80 and the genre to jazz. Enjoy the music!


In [19]:
response = await azure_openai_agent.generate_text_async(prompt)

print(response.contents[0].text)

[2025-01-02 19:31:13] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, prefer to jazz music with 80 BPM.\n\nCapabilities:\n- start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n- change_bpm: Change the BPM of the music.\n- change_music: Change the music genre and BPM.\n')]
[2025-01-02 19:31:13] INFO | root: Generating text
[2025-01-02 19:31:15] DEBUG | Response: [TextResponse(text='### Plan to Fulfill the Requirement\n\n1. **Start Music**: Invoke the `start_music` tool to begin playing music. This will randomly select a genre and BPM.\n2. **Check BPM**: If the BPM is not 80, use the `change_bpm` tool to adjust the BPM to 80.\n3. **Check Genre

The music has been started with the genre jazz and an initial BPM of 120. I have successfully changed the BPM to 80 while keeping the genre as jazz. Enjoy the music!


In [20]:
streamed_response = azure_openai_agent.stream_text(prompt)

for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-02 19:31:24] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, prefer to jazz music with 80 BPM.\n\nCapabilities:\n- start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n- change_bpm: Change the BPM of the music.\n- change_music: Change the music genre and BPM.\n')]
[2025-01-02 19:31:24] INFO | root: Generating text
[2025-01-02 19:31:26] DEBUG | Response: [TextResponse(text='### Plan to Fulfill the Requirement\n\n1. **Start Music**: Invoke the `start_music` tool to begin playing music. This will randomly select a genre and BPM.\n2. **Check BPM**: If the BPM is not 80, use the `change_bpm` tool to adjust the BPM to 80.\n3. **Check Genre

{}


[2025-01-02 19:31:26] DEBUG | Response: [ToolCallResponse(name='start_music', args='{}', call_id='call_XFgbST2W5WsODBSWcaqJMiya')]
[2025-01-02 19:31:26] INFO | Running tool: start_music
[2025-01-02 19:31:26] INFO | Tool: start_music successfully ran.
[2025-01-02 19:31:26] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: classical, BPM: 120, Volume: 0.9883905653336957', error=None, call_id='call_XFgbST2W5WsODBSWcaqJMiya', args='{}', name='start_music')]
[2025-01-02 19:31:26] INFO | root: Generating text


{"ne
{"new_bpm
{"new_bpm": 80}
{"ge
{"genre":
{"genre": "jazz
{"genre": "jazz", "
{"genre": "jazz", "bpm":
{"genre": "jazz", "bpm": 80}


[2025-01-02 19:31:27] DEBUG | Response: [ToolCallResponse(name='change_bpm', args='{"new_bpm": 80}', call_id='call_k7mFr8Cr5ZuzmCqjJUh49Ly4'), ToolCallResponse(name='change_music', args='{"genre": "jazz", "bpm": 80}', call_id='call_cK81Ih06OdV0VnazT62HozWg')]
[2025-01-02 19:31:27] INFO | Running tool: change_bpm
[2025-01-02 19:31:27] INFO | Tool: change_bpm successfully ran.
[2025-01-02 19:31:27] INFO | Running tool: change_music
[2025-01-02 19:31:27] INFO | Tool: change_music successfully ran.
[2025-01-02 19:31:27] DEBUG | Prompt: [ToolUsePrompt(output='Changing BPM to 80', error=None, call_id='call_k7mFr8Cr5ZuzmCqjJUh49Ly4', args='{"new_bpm": 80}', name='change_bpm'), ToolUsePrompt(output='Changing music to jazz with BPM 80', error=None, call_id='call_cK81Ih06OdV0VnazT62HozWg', args='{"genre": "jazz", "bpm": 80}', name='change_music')]
[2025-01-02 19:31:27] INFO | root: Generating text


The
The music
The music has
The music has been
The music has been started
The music has been started with
The music has been started with a
The music has been started with a genre
The music has been started with a genre of
The music has been started with a genre of classical
The music has been started with a genre of classical and
The music has been started with a genre of classical and a
The music has been started with a genre of classical and a BPM
The music has been started with a genre of classical and a BPM of
The music has been started with a genre of classical and a BPM of 
The music has been started with a genre of classical and a BPM of 120
The music has been started with a genre of classical and a BPM of 120.
The music has been started with a genre of classical and a BPM of 120. I
The music has been started with a genre of classical and a BPM of 120. I have
The music has been started with a genre of classical and a BPM of 120. I have successfully
The music has been started wi

[2025-01-02 19:31:29] DEBUG | Response: [TextResponse(text='The music has been started with a genre of classical and a BPM of 120. I have successfully changed the music to jazz and adjusted the BPM to 80. Enjoy the jazz music!')]


The music has been started with a genre of classical and a BPM of 120. I have successfully changed the music to jazz and adjusted the BPM to 80. Enjoy the jazz music!


In [21]:
streamed_response = openai_agent.stream_text_async(prompt)

async for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-02 19:31:31] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, prefer to jazz music with 80 BPM.\n\nCapabilities:\n- start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n- change_bpm: Change the BPM of the music.\n- change_music: Change the music genre and BPM.\n')]
[2025-01-02 19:31:31] INFO | root: Generating text
[2025-01-02 19:31:34] DEBUG | Response: [TextResponse(text='### Plan to Fulfill the Requirement\n\n1. **Start Music**: Invoke the `start_music` tool to begin playing music. This will randomly select a genre and BPM.\n2. **Check BPM**: If the BPM is not 80, use the `change_bpm` tool to adjust the BPM to 80.\n3. **Check Genre

{}


[2025-01-02 19:31:35] DEBUG | Response: [ToolCallResponse(name='start_music', args='{}', call_id='call_RuQ53vrYOdN09onAfb9AxiKU')]
[2025-01-02 19:31:35] INFO | Running tool: start_music
[2025-01-02 19:31:35] INFO | Tool: start_music successfully ran.
[2025-01-02 19:31:35] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: hip-hop, BPM: 120, Volume: 0.5555724818064931', error=None, call_id='call_RuQ53vrYOdN09onAfb9AxiKU', args='{}', name='start_music')]
[2025-01-02 19:31:35] INFO | root: Generating text


{"ne
{"new_bpm
{"new_bpm": 80}
{"ge
{"genre":
{"genre": "jazz
{"genre": "jazz", "
{"genre": "jazz", "bpm":
{"genre": "jazz", "bpm": 80}


[2025-01-02 19:31:36] DEBUG | Response: [ToolCallResponse(name='change_bpm', args='{"new_bpm": 80}', call_id='call_hrtC4hCvptk3tRQ2Do04EEKW'), ToolCallResponse(name='change_music', args='{"genre": "jazz", "bpm": 80}', call_id='call_LWezq0OA8I0cusyTwAzVJyfn')]
[2025-01-02 19:31:36] INFO | Running tool: change_bpm
[2025-01-02 19:31:36] INFO | Tool: change_bpm successfully ran.
[2025-01-02 19:31:36] INFO | Running tool: change_music
[2025-01-02 19:31:36] INFO | Tool: change_music successfully ran.
[2025-01-02 19:31:36] DEBUG | Prompt: [ToolUsePrompt(output='Changing BPM to 80', error=None, call_id='call_hrtC4hCvptk3tRQ2Do04EEKW', args='{"new_bpm": 80}', name='change_bpm'), ToolUsePrompt(output='Changing music to jazz with BPM 80', error=None, call_id='call_LWezq0OA8I0cusyTwAzVJyfn', args='{"genre": "jazz", "bpm": 80}', name='change_music')]
[2025-01-02 19:31:36] INFO | root: Generating text


The
The music
The music has
The music has been
The music has been started
The music has been started with
The music has been started with a
The music has been started with a genre
The music has been started with a genre of
The music has been started with a genre of hip
The music has been started with a genre of hip-hop
The music has been started with a genre of hip-hop and
The music has been started with a genre of hip-hop and a
The music has been started with a genre of hip-hop and a BPM
The music has been started with a genre of hip-hop and a BPM of
The music has been started with a genre of hip-hop and a BPM of 
The music has been started with a genre of hip-hop and a BPM of 120
The music has been started with a genre of hip-hop and a BPM of 120.
The music has been started with a genre of hip-hop and a BPM of 120. I
The music has been started with a genre of hip-hop and a BPM of 120. I have
The music has been started with a genre of hip-hop and a BPM of 120. I have successfully
The 

[2025-01-02 19:31:38] DEBUG | Response: [TextResponse(text='The music has been started with a genre of hip-hop and a BPM of 120. I have successfully changed the BPM to 80 and the genre to jazz. Enjoy the music!')]


The music has been started with a genre of hip-hop and a BPM of 120. I have successfully changed the BPM to 80 and the genre to jazz. Enjoy the music!


### Gemini on Google AI Studio

In [22]:
response = google_agent.generate_text(prompt)

print(response.contents[0].text)

[2025-01-02 19:31:41] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, prefer to jazz music with 80 BPM.\n\nCapabilities:\n- start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n- change_bpm: Change the BPM of the music.\n- change_music: Change the music genre and BPM.\n')]
[2025-01-02 19:31:41] INFO | root: Generating text
[2025-01-02 19:31:42] DEBUG | Response: [TextResponse(text="Okay, here's the plan:\n\n1. **Initial Music Start:** Use the `start_music` tool to begin playing music. This will give us a baseline to work from.\n2. **Change Genre and BPM:** Use the `change_music` tool to change the music to jazz and set the BPM to 80.\n3. **Co

Okay, I've started the music and then changed it to jazz with a BPM of 80 as you requested.


In [23]:
response = await google_agent.generate_text_async(prompt)

print(response.contents[0].text)

[2025-01-02 19:31:48] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, prefer to jazz music with 80 BPM.\n\nCapabilities:\n- start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n- change_bpm: Change the BPM of the music.\n- change_music: Change the music genre and BPM.\n')]
[2025-01-02 19:31:48] INFO | root: Generating text
[2025-01-02 19:31:50] DEBUG | Response: [TextResponse(text="Okay, here's the plan:\n\n1. **Initial Music Start:** Use the `start_music` tool to begin playing music. This will give us a baseline to work from.\n2. **Change Genre and BPM:** Use the `change_music` tool to change the music to jazz and set the BPM to 80.\n3. **Co

Okay, I've started the music and then changed it to jazz with a BPM of 80. Enjoy!


In [26]:
streamed_response = google_agent.stream_text(prompt)

for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-02 19:33:14] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, prefer to jazz music with 80 BPM.\n\nCapabilities:\n- start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n- change_bpm: Change the BPM of the music.\n- change_music: Change the music genre and BPM.\n')]
[2025-01-02 19:33:14] INFO | root: Generating text
[2025-01-02 19:33:16] DEBUG | Response: [TextResponse(text="Okay, here's the plan:\n\n1. **Initial Music Start:** Use the `start_music` tool to begin playing music. This will give us a baseline to work from.\n2. **Change Genre and BPM:** Use the `change_music` tool to change the music to jazz and set the BPM to 80.\n3. **Co

{}


[2025-01-02 19:33:17] DEBUG | Response: [ToolCallResponse(name='start_music', args='{}', call_id='6D6YTTx8D6WS0c4ZWvs039c7')]
[2025-01-02 19:33:17] INFO | Running tool: start_music
[2025-01-02 19:33:17] INFO | Tool: start_music successfully ran.
[2025-01-02 19:33:17] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: pop, BPM: 120, Volume: 0.5813511588061258', error=None, call_id='6D6YTTx8D6WS0c4ZWvs039c7', args='{}', name='start_music')]
[2025-01-02 19:33:17] INFO | root: Generating text


{"bpm": 80, "genre": "jazz"}


[2025-01-02 19:33:18] DEBUG | Response: [ToolCallResponse(name='change_music', args='{"bpm": 80, "genre": "jazz"}', call_id='aU1d1dbCKBl9xu1BGixjIYQy')]
[2025-01-02 19:33:18] INFO | Running tool: change_music
[2025-01-02 19:33:18] INFO | Tool: change_music successfully ran.
[2025-01-02 19:33:18] DEBUG | Prompt: [ToolUsePrompt(output='Changing music to jazz with BPM 80', error=None, call_id='aU1d1dbCKBl9xu1BGixjIYQy', args='{"bpm": 80, "genre": "jazz"}', name='change_music')]
[2025-01-02 19:33:18] INFO | root: Generating text


Okay
Okay, I've started the music and then changed it to jazz with a BPM
Okay, I've started the music and then changed it to jazz with a BPM of 80. Enjoy!

Okay, I've started the music and then changed it to jazz with a BPM of 80. Enjoy!



In [27]:
streamed_response = google_agent.stream_text_async(prompt)

async for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-02 19:33:24] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, prefer to jazz music with 80 BPM.\n\nCapabilities:\n- start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n- change_bpm: Change the BPM of the music.\n- change_music: Change the music genre and BPM.\n')]
[2025-01-02 19:33:24] INFO | root: Generating text
[2025-01-02 19:33:26] DEBUG | Response: [TextResponse(text="Okay, here's the plan:\n\n1. **Initial Music Start:** Use the `start_music` tool to begin playing music. This will give us a baseline to work from.\n2. **Change Genre and BPM:** Use the `change_music` tool to change the music to jazz and set the BPM to 80.\n3. **Co

{}


[2025-01-02 19:33:27] DEBUG | Response: [ToolCallResponse(name='start_music', args='{}', call_id='3X0Y7UP9dDAzvlYEH9cpbgRK')]
[2025-01-02 19:33:27] INFO | Running tool: start_music
[2025-01-02 19:33:27] INFO | Tool: start_music successfully ran.
[2025-01-02 19:33:27] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: rock, BPM: 120, Volume: 0.9754679890395785', error=None, call_id='3X0Y7UP9dDAzvlYEH9cpbgRK', args='{}', name='start_music')]
[2025-01-02 19:33:27] INFO | root: Generating text


{"genre": "jazz", "bpm": 80}


[2025-01-02 19:33:28] DEBUG | Response: [ToolCallResponse(name='change_music', args='{"genre": "jazz", "bpm": 80}', call_id='FQNNaYt7ZWusCcbn89KLzx8S')]
[2025-01-02 19:33:28] INFO | Running tool: change_music
[2025-01-02 19:33:28] INFO | Tool: change_music successfully ran.
[2025-01-02 19:33:28] DEBUG | Prompt: [ToolUsePrompt(output='Changing music to jazz with BPM 80', error=None, call_id='FQNNaYt7ZWusCcbn89KLzx8S', args='{"genre": "jazz", "bpm": 80}', name='change_music')]
[2025-01-02 19:33:28] INFO | root: Generating text


Okay
Okay, I've started the music and then changed it to jazz with a BPM
Okay, I've started the music and then changed it to jazz with a BPM of 80. Enjoy!

Okay, I've started the music and then changed it to jazz with a BPM of 80. Enjoy!



### Gemini on VertexAI

In [28]:
response = vertexai_agent.generate_text(prompt)

print(response.contents[0].text)

[2025-01-02 19:33:50] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, prefer to jazz music with 80 BPM.\n\nCapabilities:\n- start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n- change_bpm: Change the BPM of the music.\n- change_music: Change the music genre and BPM.\n')]
[2025-01-02 19:33:50] INFO | root: Generating text
[2025-01-02 19:33:53] DEBUG | Response: [TextResponse(text="Okay, here's the plan:\n\n1. **Initial Music Start:** Use the `start_music` tool to begin playing music. This will give us a baseline to work from.\n2. **Change Genre and BPM:** Use the `change_music` tool to change the music to jazz and set the BPM to 80.\n3. **Co

Okay, I've started the music and then changed it to jazz with a BPM of 80. Enjoy!


In [29]:
response = await vertexai_agent.generate_text_async(prompt)

print(response.contents[0].text)

[2025-01-02 19:33:58] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, prefer to jazz music with 80 BPM.\n\nCapabilities:\n- start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n- change_bpm: Change the BPM of the music.\n- change_music: Change the music genre and BPM.\n')]
[2025-01-02 19:33:58] INFO | root: Generating text
[2025-01-02 19:34:00] DEBUG | Response: [TextResponse(text="Okay, here's the plan:\n\n1. **Initial Music Start:** Use the `start_music` tool to begin playing music. This will give us a baseline to work from.\n2. **Change Genre and BPM:** Use the `change_music` tool to change the music to jazz and set the BPM to 80.\n3. **Co

Okay, I've started the music and then changed it to jazz with a BPM of 80. Enjoy!


In [31]:
streamed_response = vertexai_agent.stream_text(prompt)

for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-02 19:35:04] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, prefer to jazz music with 80 BPM.\n\nCapabilities:\n- start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n- change_bpm: Change the BPM of the music.\n- change_music: Change the music genre and BPM.\n')]
[2025-01-02 19:35:04] INFO | root: Generating text
[2025-01-02 19:35:06] DEBUG | Response: [TextResponse(text="Okay, here's the plan:\n\n1. **Initial Music Start:** Use the `start_music` tool to begin playing music. This will give us a baseline to work from.\n2. **Change Genre and BPM:** Use the `change_music` tool to change the music to jazz and set the BPM to 80.\n3. **Co

{}


[2025-01-02 19:35:07] DEBUG | Response: [ToolCallResponse(name='start_music', args='{}', call_id='wYlRWgtqsyQB3rNVfiEr0Zj4')]
[2025-01-02 19:35:07] INFO | Running tool: start_music
[2025-01-02 19:35:07] INFO | Tool: start_music successfully ran.
[2025-01-02 19:35:07] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: hip-hop, BPM: 120, Volume: 0.6704972252283417', error=None, call_id='wYlRWgtqsyQB3rNVfiEr0Zj4', args='{}', name='start_music')]
[2025-01-02 19:35:07] INFO | root: Generating text


{"genre": "jazz", "bpm": 80}


[2025-01-02 19:35:09] DEBUG | Response: [ToolCallResponse(name='change_music', args='{"genre": "jazz", "bpm": 80}', call_id='glmihc4QJaDRtroUvoLR2NBF')]
[2025-01-02 19:35:09] INFO | Running tool: change_music
[2025-01-02 19:35:09] INFO | Tool: change_music successfully ran.
[2025-01-02 19:35:09] DEBUG | Prompt: [ToolUsePrompt(output='Changing music to jazz with BPM 80', error=None, call_id='glmihc4QJaDRtroUvoLR2NBF', args='{"genre": "jazz", "bpm": 80}', name='change_music')]
[2025-01-02 19:35:09] INFO | root: Generating text


Okay
Okay, I've started the music and then changed it to jazz with a BPM
Okay, I've started the music and then changed it to jazz with a BPM of 80. Enjoy!

Okay, I've started the music and then changed it to jazz with a BPM of 80. Enjoy!



In [32]:
streamed_response = vertexai_agent.stream_text_async(prompt)

async for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-02 19:35:13] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, prefer to jazz music with 80 BPM.\n\nCapabilities:\n- start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n- change_bpm: Change the BPM of the music.\n- change_music: Change the music genre and BPM.\n')]
[2025-01-02 19:35:13] INFO | root: Generating text
[2025-01-02 19:35:15] DEBUG | Response: [TextResponse(text="Okay, here's the plan:\n\n1. **Initial Music Start:** Use the `start_music` tool to begin playing music. This will give us a baseline to work from.\n2. **Change Genre and BPM:** Use the `change_music` tool to change the music to jazz and set the BPM to 80.\n3. **Co

{}


[2025-01-02 19:35:17] DEBUG | Response: [ToolCallResponse(name='start_music', args='{}', call_id='B9Mcrbpqm5bix7EUWmKwaPrc')]
[2025-01-02 19:35:17] INFO | Running tool: start_music
[2025-01-02 19:35:17] INFO | Tool: start_music successfully ran.
[2025-01-02 19:35:17] DEBUG | Prompt: [ToolUsePrompt(output='Starting music! Genre: hip-hop, BPM: 120, Volume: 0.6564488142194529', error=None, call_id='B9Mcrbpqm5bix7EUWmKwaPrc', args='{}', name='start_music')]
[2025-01-02 19:35:17] INFO | root: Generating text


{"genre": "jazz", "bpm": 80}


[2025-01-02 19:35:18] DEBUG | Response: [ToolCallResponse(name='change_music', args='{"genre": "jazz", "bpm": 80}', call_id='GSL4LI156jdRVYKtxfT2G4vj')]
[2025-01-02 19:35:18] INFO | Running tool: change_music
[2025-01-02 19:35:18] INFO | Tool: change_music successfully ran.
[2025-01-02 19:35:18] DEBUG | Prompt: [ToolUsePrompt(output='Changing music to jazz with BPM 80', error=None, call_id='GSL4LI156jdRVYKtxfT2G4vj', args='{"genre": "jazz", "bpm": 80}', name='change_music')]
[2025-01-02 19:35:18] INFO | root: Generating text


Okay
Okay, I've started the music and then changed it to jazz with a BPM
Okay, I've started the music and then changed it to jazz with a BPM of 80. Enjoy!

Okay, I've started the music and then changed it to jazz with a BPM of 80. Enjoy!



### Claude on Anthropic

In [33]:
response = claude_agent.generate_text(prompt)

print(response.contents[0].text)

[2025-01-02 19:35:22] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, prefer to jazz music with 80 BPM.\n\nCapabilities:\n- start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n- change_bpm: Change the BPM of the music.\n- change_music: Change the music genre and BPM.\n')]
[2025-01-02 19:35:22] INFO | root: Generating text
[2025-01-02 19:35:25] DEBUG | Response: [TextResponse(text="Here's a concise plan to fulfill the requirement:\n\n1. Use the change_music tool to set the genre to jazz and BPM to 80.\n2. If change_music doesn't allow specifying both genre and BPM simultaneously:\n   a. Use start_music to begin playing music.\n   b. Use chang

Great! The music has been changed to jazz with a BPM of 80, exactly as requested. The system has confirmed that it's changing the music to jazz with a BPM of 80. 

To summarize:
1. Jazz music is now playing.
2. The tempo is set to 80 beats per minute (BPM).

This fulfills the requirement to "Play some music, well, prefer to jazz music with 80 BPM." Is there anything else you'd like to adjust with the music, or do you have any other requests?


In [34]:
response = await claude_agent.generate_text_async(prompt)

print(response.contents[0].text)

[2025-01-02 19:35:33] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, prefer to jazz music with 80 BPM.\n\nCapabilities:\n- start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n- change_bpm: Change the BPM of the music.\n- change_music: Change the music genre and BPM.\n')]
[2025-01-02 19:35:33] INFO | root: Generating text
[2025-01-02 19:35:36] DEBUG | Response: [TextResponse(text="Here's a concise plan to fulfill the requirement:\n\n1. Use the change_music tool to set the genre to jazz and BPM to 80.\n2. If change_music doesn't allow specifying both genre and BPM simultaneously:\n   a. Use start_music to begin playing music.\n   b. Use chang

Great! The music has been changed to jazz with a BPM of 80, exactly as requested. The system has confirmed that it's changing the music to jazz with a BPM of 80. 

To summarize:
1. Jazz music is now playing.
2. The tempo is set to 80 beats per minute (BPM).

This fulfills the requirement to "Play some music, well, prefer to jazz music with 80 BPM." Is there anything else you'd like to adjust with the music, or do you have any other requests?


In [35]:
streamed_response = claude_agent.stream_text(prompt)

for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-02 19:35:48] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, prefer to jazz music with 80 BPM.\n\nCapabilities:\n- start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n- change_bpm: Change the BPM of the music.\n- change_music: Change the music genre and BPM.\n')]
[2025-01-02 19:35:48] INFO | root: Generating text
[2025-01-02 19:35:50] DEBUG | Response: [TextResponse(text="Here's a concise plan to fulfill the requirement:\n\n1. Use the change_music tool to set the genre to jazz and BPM to 80.\n2. If change_music is successful, we're done.\n3. If change_music fails or is not available:\n   a. Use start_music to turn on music.\n   b. U

Certainly! I'll
Certainly! I'll put the plan into action by
Certainly! I'll put the plan into action by using the change_music tool to set the genre to
Certainly! I'll put the plan into action by using the change_music tool to set the genre to jazz and the BPM to
Certainly! I'll put the plan into action by using the change_music tool to set the genre to jazz and the BPM to 80. Here's the function call:
{"genre
{"genre": "jazz"
{"genre": "jazz", "bpm":
{"genre": "jazz", "bpm": 80}


[2025-01-02 19:35:54] DEBUG | Response: [TextResponse(text="Certainly! I'll put the plan into action by using the change_music tool to set the genre to jazz and the BPM to 80. Here's the function call:"), ToolCallResponse(name='change_music', args='{"genre": "jazz", "bpm": 80}', call_id='toolu_012ydYqekqf5CM2mhqvdU2w6')]
[2025-01-02 19:35:54] INFO | Running tool: change_music
[2025-01-02 19:35:54] INFO | Tool: change_music successfully ran.
[2025-01-02 19:35:54] DEBUG | Prompt: [ToolUsePrompt(output='Changing music to jazz with BPM 80', error=None, call_id='toolu_012ydYqekqf5CM2mhqvdU2w6', args='{"genre": "jazz", "bpm": 80}', name='change_music')]
[2025-01-02 19:35:54] INFO | root: Generating text


Great! The change
Great! The change_music function has been successfully calle
Great! The change_music function has been successfully called, and the music has been changed to jazz
Great! The change_music function has been successfully called, and the music has been changed to jazz with a BPM of 80, exactly
Great! The change_music function has been successfully called, and the music has been changed to jazz with a BPM of 80, exactly as requested. This fulfills the user
Great! The change_music function has been successfully called, and the music has been changed to jazz with a BPM of 80, exactly as requested. This fulfills the user's requirement to "Play some music, well
Great! The change_music function has been successfully called, and the music has been changed to jazz with a BPM of 80, exactly as requested. This fulfills the user's requirement to "Play some music, well, prefer to jazz music with 80 B
Great! The change_music function has been successfully called, and the music has bee

[2025-01-02 19:35:57] DEBUG | Response: [TextResponse(text='Great! The change_music function has been successfully called, and the music has been changed to jazz with a BPM of 80, exactly as requested. This fulfills the user\'s requirement to "Play some music, well, prefer to jazz music with 80 BPM."\n\nTo summarize:\n1. The music genre has been set to jazz.\n2. The tempo (BPM) has been set to 80.\n\nIs there anything else you\'d like to do with the music or any other requests?')]


Great! The change_music function has been successfully called, and the music has been changed to jazz with a BPM of 80, exactly as requested. This fulfills the user's requirement to "Play some music, well, prefer to jazz music with 80 BPM."

To summarize:
1. The music genre has been set to jazz.
2. The tempo (BPM) has been set to 80.

Is there anything else you'd like to do with the music or any other requests?


In [36]:
streamed_response = claude_agent.stream_text_async(prompt)

async for chunk in streamed_response:
    for content in chunk.contents:
        if isinstance(content, TextResponse):
            print(content.text, flush=True)
        elif isinstance(content, ToolCallResponse):
            print(content.args, flush=True)

[2025-01-02 19:36:00] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, prefer to jazz music with 80 BPM.\n\nCapabilities:\n- start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n- change_bpm: Change the BPM of the music.\n- change_music: Change the music genre and BPM.\n')]
[2025-01-02 19:36:00] INFO | root: Generating text
[2025-01-02 19:36:03] DEBUG | Response: [TextResponse(text="Here's a concise plan to fulfill the requirement:\n\n1. Use the change_music tool to set the genre to jazz and BPM to 80.\n2. If change_music is successful, we're done.\n3. If change_music fails or is not available:\n   a. Use start_music to turn on music.\n   b. U

Certainly! I'll
Certainly! I'll put the plan into action by using the change
Certainly! I'll put the plan into action by using the change_music tool to set the genre to
Certainly! I'll put the plan into action by using the change_music tool to set the genre to jazz and the BPM to
Certainly! I'll put the plan into action by using the change_music tool to set the genre to jazz and the BPM to 80. Here's the function call:
{"genre"
{"genre": "jazz
{"genre": "jazz"
{"genre": "jazz", "bpm"
{"genre": "jazz", "bpm": 80}


[2025-01-02 19:36:07] DEBUG | Response: [TextResponse(text="Certainly! I'll put the plan into action by using the change_music tool to set the genre to jazz and the BPM to 80. Here's the function call:"), ToolCallResponse(name='change_music', args='{"genre": "jazz", "bpm": 80}', call_id='toolu_018WGPMh1XdZ7rV41onSV2gQ')]
[2025-01-02 19:36:07] INFO | Running tool: change_music
[2025-01-02 19:36:07] INFO | Tool: change_music successfully ran.
[2025-01-02 19:36:07] DEBUG | Prompt: [ToolUsePrompt(output='Changing music to jazz with BPM 80', error=None, call_id='toolu_018WGPMh1XdZ7rV41onSV2gQ', args='{"genre": "jazz", "bpm": 80}', name='change_music')]
[2025-01-02 19:36:07] INFO | root: Generating text


Great! The change
Great! The change_music function has been successfully called, and the music
Great! The change_music function has been successfully called, and the music has been changed to jazz with a BPM
Great! The change_music function has been successfully called, and the music has been changed to jazz with a BPM of 80, exactly as requested.
Great! The change_music function has been successfully called, and the music has been changed to jazz with a BPM of 80, exactly as requested. This fulfills the user's requirement to
Great! The change_music function has been successfully called, and the music has been changed to jazz with a BPM of 80, exactly as requested. This fulfills the user's requirement to "Play some music, well
Great! The change_music function has been successfully called, and the music has been changed to jazz with a BPM of 80, exactly as requested. This fulfills the user's requirement to "Play some music, well, prefer to jazz music with 80 B
Great! The change_music fu

[2025-01-02 19:36:10] DEBUG | Response: [TextResponse(text='Great! The change_music function has been successfully called, and the music has been changed to jazz with a BPM of 80, exactly as requested. This fulfills the user\'s requirement to "Play some music, well, prefer to jazz music with 80 BPM."\n\nTo summarize what has been done:\n1. We used the change_music tool to set the genre to jazz and the BPM to 80.\n2. The function was successful, so no further actions were needed.\n\nThe music is now playing jazz at 80 BPM, which should create a nice, relaxed atmosphere. Is there anything else you\'d like to adjust with the music?')]


Great! The change_music function has been successfully called, and the music has been changed to jazz with a BPM of 80, exactly as requested. This fulfills the user's requirement to "Play some music, well, prefer to jazz music with 80 BPM."

To summarize what has been done:
1. We used the change_music tool to set the genre to jazz and the BPM to 80.
2. The function was successful, so no further actions were needed.

The music is now playing jazz at 80 BPM, which should create a nice, relaxed atmosphere. Is there anything else you'd like to adjust with the music?


# Structured output using tool calling

## Response schema as a tool

Structured output in langrila is just tool calling (please refer to [01.introduction.ipynb](./01.introduction.ipynb)), so you can get structured response even if you are using tool calling.

In [37]:
from enum import Enum

from pydantic import BaseModel, Field


class DiscoBallSchema(BaseModel):
    power: bool = Field(..., description="Whether to power the disco ball.")
    spinning: bool = Field(..., description="Whether the disco ball is spinning.")


class MusicGenre(str, Enum):
    rock = "rock"
    pop = "pop"
    jazz = "jazz"
    classical = "classical"
    hip_hop = "hip-hop"


class MusicSchema(BaseModel):
    genre: MusicGenre = Field(
        ...,
        description="The genre of music to play.",
    )
    bpm: int = Field(
        ...,
        description="The BPM of the music.",
        ge=60,
        le=180,
    )
    volume: float = Field(
        ...,
        description="The volume level to set the music to.",
        ge=0,
        le=1,
    )

In [38]:
openai_agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
    tools=tools,
    planning=True,
    response_schema_as_tool=MusicSchema,
)

In [39]:
prompt = "Play some music, well, prefer to jazz music with 80 BPM."

response = openai_agent.generate_text(prompt)

MusicSchema.model_validate_json(response.contents[0].text)

[2025-01-02 19:36:29] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, prefer to jazz music with 80 BPM.\n\nCapabilities:\n- start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n- change_bpm: Change the BPM of the music.\n- change_music: Change the music genre and BPM.\n')]
[2025-01-02 19:36:29] INFO | root: Generating text
[2025-01-02 19:36:32] DEBUG | Response: [TextResponse(text='### Plan to Address the Requirement\n\n1. **Start Music**: Invoke the `start_music` tool to begin playing music. This will randomly select a genre and BPM.\n2. **Check BPM**: If the BPM is not 80, use the `change_bpm` tool to adjust the BPM to 80.\n3. **Check Genre

MusicSchema(genre=<MusicGenre.jazz: 'jazz'>, bpm=80, volume=0.7410422055578767)

Same for other clients, so examples will be omitted.

## Native response schema

If you want to use native structured output, you can specify the original parameters of LLM provider API. For example, for OpenAI API, you can use `response_format` parameter supported by OpenAI API natively.

In [40]:
from enum import Enum

from pydantic import BaseModel, Field


class NativeMusicGenre(str, Enum):
    rock = "rock"
    pop = "pop"
    jazz = "jazz"
    classical = "classical"
    hip_hop = "hip-hop"


class NativeMusicSchema(BaseModel):
    genre: NativeMusicGenre = Field(
        ...,
        description="The genre of music to play.",
    )
    bpm: int = Field(
        ...,
        description="The BPM of the music.",
        # Not supported below
        # ge=60,
        # le=180,
    )
    volume: float = Field(
        ...,
        description="The volume level to set the music to.",
        # Not supported below
        # ge=0,
        # le=1,
    )

In [41]:
openai_agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    tools=tools,
    planning=True,
    response_format=NativeMusicSchema,
)

In [42]:
prompt = "Play some music, well, prefer to jazz music with 80 BPM."

response = openai_agent.generate_text(prompt)

NativeMusicSchema.model_validate_json(response.contents[0].text)

[2025-01-02 19:37:04] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPlay some music, well, prefer to jazz music with 80 BPM.\n\nCapabilities:\n- start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n- change_bpm: Change the BPM of the music.\n- change_music: Change the music genre and BPM.\n')]
[2025-01-02 19:37:04] INFO | root: Generating text
[2025-01-02 19:37:06] DEBUG | Response: [TextResponse(text="### Plan to Fulfill the Requirement:\n\n1. **Start Music**: Invoke the `start_music` tool to initiate playing some music.\n2. **Change Music Genre**: If the genre is not jazz, use the `change_music` tool to set the genre to jazz and BPM to 80. If the BPM is not 80,

NativeMusicSchema(genre=<NativeMusicGenre.jazz: 'jazz'>, bpm=80, volume=0.2042542640891518)

In [43]:
response

Response(type='Response', role='assistant', contents=[TextResponse(text='{"genre":"jazz","bpm":80,"volume":0.2042542640891518}')], usage=<langrila.core.usage.NamedUsage object at 0x7f766e6f9720>, raw=ParsedChatCompletion[NativeMusicSchema](id='chatcmpl-AlCrbjrMU2oPjOR3UxpAHLdwH8Xz6', choices=[ParsedChoice[NativeMusicSchema](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[NativeMusicSchema](content='{"genre":"jazz","bpm":80,"volume":0.2042542640891518}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[], parsed=NativeMusicSchema(genre=<NativeMusicGenre.jazz: 'jazz'>, bpm=80, volume=0.2042542640891518)))], created=1735814227, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_0aa8d3e20b', usage=CompletionUsage(completion_tokens=29, prompt_tokens=628, total_tokens=657, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens

# Dependency injection

While tools can be passed to an Agent directly as functions, using the Tool class allows for more flexible control. One example of this flexibility is dependency injection. 

Standard tool calling requires the LLM to generate all arguments specified as 'required'. For arguments not specified as 'required', either a default value or a value generated by the LLM is used.

With the Tool class, you can use a 'context' argument to separate parameters into those generated by the LLM and those injected from an external source. This notebook demonstrates an example of this approach.

Here is an example of tool invokes an other agent. 

In [44]:
def run_music_agent(
    agent: Agent,  # type: ignore
    agent_name: str,
    instruction: str,
) -> str:
    """
    This function is used to run the agent which has the capabilities to manage the music settings.
    Agent can use the following tools:

    Tools:
    start_music: Turn on the music. The genre, BPM, and volume are randomly selected.
    change_bpm: Change the BPM of the music.
    change_music: Change the music genre and BPM.

    Parameters
    ----------
    agent : Agent
        The agent instance.
    agent_name : str
        The name of the agent.
    instruction : str
        The detail and specific instruction to the agent, including the plan to get answer.

    Returns
    ----------
    str
        The response from the agent.
    """
    if not isinstance(agent, Agent):
        raise ValueError(
            "Subagent must be an instance of Agent class. "
            "Please provide the correct agent instance."
        )

    return agent.generate_text(instruction, name=agent_name).contents[0].text  # type: ignore

To show tha example of dependency injection, two agents are defined; `music_agent` for managing music setting, and another agent calls `music_agent`.

In [45]:
# The agent for managing music settings
music_agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
    tools=[start_music, change_bpm, change_music],
)

# The main agent to call the music agent
agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    tools=[
        Tool(tool=run_music_agent, context={"agent": music_agent, "agent_name": "music_agent"}),
    ],
)

The tool's parameters specified in the context arguments of Tool instance are injected externally. In this case, `agent` argument and `agent_name` argument of the run_music_agent tool are injected.

In [46]:
prompt = "Play some music, well, prefer to jazz music with 120 BPM."

response = agent.generate_text(prompt)

[2025-01-02 19:45:41] DEBUG | Prompt: [TextPrompt(text='Play some music, well, prefer to jazz music with 120 BPM.')]
[2025-01-02 19:45:41] INFO | root: Generating text
[2025-01-02 19:45:42] DEBUG | Response: [ToolCallResponse(name='run_music_agent', args='{"instruction":"Start playing jazz music with a BPM of 120."}', call_id='call_lU0SSUm8nGfeOcwHbUTxuGXP')]
[2025-01-02 19:45:42] INFO | Running tool: run_music_agent
[2025-01-02 19:45:42] DEBUG | Prompt: [TextPrompt(text='Start playing jazz music with a BPM of 120.')]
[2025-01-02 19:45:42] INFO | music_agent: Generating text
[2025-01-02 19:45:43] DEBUG | Response: [ToolCallResponse(name='change_music', args='{"genre":"jazz","bpm":120}', call_id='call_irwPJIZM6sLAMnfILpQ8dCzz')]
[2025-01-02 19:45:43] INFO | Running tool: change_music
[2025-01-02 19:45:43] INFO | Tool: change_music successfully ran.
[2025-01-02 19:45:43] DEBUG | Prompt: [ToolUsePrompt(output='Changing music to jazz with BPM 120', error=None, call_id='call_irwPJIZM6sLAMnf

As you can see in the log messages above, the agent instance invoked music_agent, but the parameters created by the agent to call `music_agent` is only `instruction` argument while `agent` argument and `agent_name` are injected. The arguments are also validated when the tool is called. If you passed wrong value for a context parameter, the error will occur when run tool.

In [48]:
from langrila import AgentConfig

# Define not to retry the request even if an error occurs
agent_config = AgentConfig(
    max_error_retries=1,
)

# The main agent to call the music agent
agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    tools=[
        Tool(
            tool=run_music_agent,
            # wrong for agent argument below
            context={"agent": "wrong_value", "agent_name": "music_agent"},
        ),
    ],
    agent_config=agent_config,
)

In [49]:
prompt = "Play some music, well, prefer to jazz music with 120 BPM."

response = agent.generate_text(prompt)

[2025-01-02 19:49:12] DEBUG | Prompt: [TextPrompt(text='Play some music, well, prefer to jazz music with 120 BPM.')]
[2025-01-02 19:49:12] INFO | root: Generating text
[2025-01-02 19:49:13] DEBUG | Response: [ToolCallResponse(name='run_music_agent', args='{"instruction":"Start playing jazz music with a tempo of 120 BPM."}', call_id='call_h3PtCGoDTGxau2k9yATBMiv9')]
[2025-01-02 19:49:13] INFO | Running tool: run_music_agent
[2025-01-02 19:49:13] ERROR | Validation error occurred while running tool. Retrying...
Tool: run_music_agent
Attribute: agent
Input value: wrong_value
ValidationError: Input should be a dictionary or an instance of Agent


RetryLimitExceededError: Retry limit exceeded. Please try again or change the request parameters.

Actually the tools calling other agent such as this are dynamically created in the Agent class if you pass the subagents parameter of the agent, but in this notebook, dared to define a tool as an example and passed it to the Agent class as a tool. For multi-agent example, Please refer to [03.multi_agent.ipynb](./03.multi_agent.ipynb)

# Tool result serializer

Tool class accepts `serializer` parameter that is callable object to convert any value to string. `str` is used in default, but you can also use custom serializer. This capability allows us to reuse the tools implemented in daily development by only implementing serializer. Let's change the example above using custom serializer.

In [50]:
import json

from langrila import Response


def run_music_agent(
    agent: Agent,  # type: ignore
    agent_name: str,
    instruction: str,
) -> Response:
    """
    This function is used to run the agent which has the capabilities to manage the music settings.
    Agent can use the following tools:

    Tools:
    start_music: Turn on the music. The genre, BPM, and volume are randomly selected.
    change_bpm: Change the BPM of the music.
    change_music: Change the music genre and BPM.

    Parameters
    ----------
    agent : Agent
        The agent instance.
    agent_name : str
        The name of the agent.
    instruction : str
        The detail and specific instruction to the agent, including the plan to get answer.

    Returns
    ----------
    Response
        The response from the agent.
    """
    if not isinstance(agent, Agent):
        raise ValueError(
            "Subagent must be an instance of Agent class. "
            "Please provide the correct agent instance."
        )

    return agent.generate_text(instruction, name=agent_name)


def response_serializer(response: Response) -> str:
    """
    Serialize the response to a string.

    Parameters
    ----------
    response : Response
        The response from the agent.

    Returns
    ----------
    str
        The serialized response.
    """
    return json.dumps(response.model_dump(include={"role", "contents", "name"}), ensure_ascii=False)

In [ ]:
from langrila import AgentConfig

# Define not to retry the request even if an error occurs
agent_config = AgentConfig(
    max_error_retries=1,
)

# The main agent to call the music agent
agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    tools=[
        Tool(
            tool=run_music_agent,
            context={"agent": music_agent, "agent_name": "music_agent"},
            serializer=response_serializer,  # serializer
        ),
    ],
)

In [52]:
prompt = "Play some music, well, prefer to jazz music with 120 BPM."

response = agent.generate_text(prompt)

[2025-01-02 19:55:14] DEBUG | Prompt: [TextPrompt(text='Play some music, well, prefer to jazz music with 120 BPM.')]
[2025-01-02 19:55:14] INFO | root: Generating text
[2025-01-02 19:55:15] DEBUG | Response: [ToolCallResponse(name='run_music_agent', args='{"instruction":"Start playing jazz music at 120 BPM."}', call_id='call_lp73gPDEWAJxg2614QHWXNGH')]
[2025-01-02 19:55:15] INFO | Running tool: run_music_agent
[2025-01-02 19:55:15] DEBUG | Prompt: [TextPrompt(text='Start playing jazz music at 120 BPM.')]
[2025-01-02 19:55:15] INFO | music_agent: Generating text
[2025-01-02 19:55:16] DEBUG | Response: [ToolCallResponse(name='change_music', args='{"genre":"jazz","bpm":120}', call_id='call_ZZJw9ENglIK6w7r1uY82bIiu')]
[2025-01-02 19:55:16] INFO | Running tool: change_music
[2025-01-02 19:55:16] INFO | Tool: change_music successfully ran.
[2025-01-02 19:55:16] DEBUG | Prompt: [ToolUsePrompt(output='Changing music to jazz with BPM 120', error=None, call_id='call_ZZJw9ENglIK6w7r1uY82bIiu', ar

ToolUsePrompt of the `run_music_agent` in the log message is changed to json string as expected.